# Tutorial 1 - SVM Classification


We will perform two prediction tasks:
1) Whether the price of an AIRBNB listing is greater than or equal to $150 (`price_gte_150` column),<br>
2) What is the price category, among 4 categories, of an AIRBNB listing (`price_category` column)

**The unit of analysis is an AIRBNB LISTING**

# Setup

In [1]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(42)


# Get the data

In [2]:
#We will predict the "price_gte_150" value in the data set:

airbnb = pd.read_csv("airbnb.csv")
airbnb.head()

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_75
3,0,0,Roslindale,42.281106,-71.121021,House,Private room,4,1.0,1.0,...,2,25,1,1,0,100.0,moderate,75,0,lte_75
4,1,1,Roslindale,42.284512,-71.136258,House,Private room,2,1.5,1.0,...,1,0,2,29,380,99.0,flexible,79,0,btw_75-150


In [3]:
airbnb

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_75
3,0,0,Roslindale,42.281106,-71.121021,House,Private room,4,1.0,1.0,...,2,25,1,1,0,100.0,moderate,75,0,lte_75
4,1,1,Roslindale,42.284512,-71.136258,House,Private room,2,1.5,1.0,...,1,0,2,29,380,99.0,flexible,79,0,btw_75-150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10267,0,1,Charlestown,42.380968,-71.083795,Apartment,Private room,3,1.0,1.0,...,1,24,2,4,344,90.0,strict,69,0,lte_75
10268,0,1,Allston,42.347503,-71.118336,Apartment,Private room,2,1.0,1.0,...,1,0,3,0,0,NaN,strict,150,1,btw_75-150
10269,0,0,Charlestown,42.371771,-71.071300,Apartment,Entire home/apt,4,1.0,1.0,...,1,0,3,0,0,NaN,flexible,198,1,btw_151-225
10270,0,1,Charlestown,42.385226,-71.080923,Apartment,Private room,2,1.0,1.0,...,1,30,1,2,8,90.0,strict,65,0,lte_75


# Split the data into train and test

In [4]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(airbnb, test_size=0.3)

## Drop the variables we can't use for the binary task

In [5]:
# We can't use the following columns in this tutorial, because they are not for binary classification tasks

train = train_set.drop(['price', 'price_category'], axis=1)
test = test_set.drop(['price', 'price_category'], axis=1)

### Be careful: we haven't seperated the target column yet

## Check the missing values

In [6]:
train_set.isna().sum()

host_is_superhost                       0
host_identity_verified                  0
neighbourhood_cleansed                  0
latitude                                0
longitude                               0
property_type                           8
room_type                               0
accommodates                            0
bathrooms                              19
bedrooms                               19
beds                                   16
bed_type                                0
Number of amenities                     0
guests_included                         0
price_per_extra_person                  0
minimum_nights                          0
number_of_reviews                       0
number_days_btw_first_last_review       0
review_scores_rating                 1609
cancellation_policy                     0
price                                   0
price_gte_150                           0
price_category                          0
dtype: int64

In [7]:
test_set.isna().sum()

host_is_superhost                      0
host_identity_verified                 0
neighbourhood_cleansed                 0
latitude                               0
longitude                              0
property_type                          1
room_type                              0
accommodates                           0
bathrooms                             17
bedrooms                              11
beds                                   8
bed_type                               0
Number of amenities                    0
guests_included                        0
price_per_extra_person                 0
minimum_nights                         0
number_of_reviews                      0
number_days_btw_first_last_review      0
review_scores_rating                 674
cancellation_policy                    0
price                                  0
price_gte_150                          0
price_category                         0
dtype: int64

# Data Prep

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable (we don't want to transform it)

In [9]:
train_y = train[['price_gte_150']]
test_y = test[['price_gte_150']]

train_inputs = train.drop(['price_gte_150'], axis=1)
test_inputs = test.drop(['price_gte_150'], axis=1)

##  Identify the numerical and categorical columns

**At this stage, you can manually identify numeric, binary, and categorical columns as follows:**

`numeric_columns = ['latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'Number of amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']`
 
 `binary_columns = ['host_is_superhost', 'host_identity_verified']`
 
 `categorical_columns = ['neighbourhood_cleansed', 'property_type', 'room_type', 'bed_type', 'cancellation_policy']`
 
<br>
 
**If you do not want to manually type these, you can do the below tricks:**

In [10]:
train_inputs.dtypes

host_is_superhost                      int64
host_identity_verified                 int64
neighbourhood_cleansed                object
latitude                             float64
longitude                            float64
property_type                         object
room_type                             object
accommodates                           int64
bathrooms                            float64
bedrooms                             float64
beds                                 float64
bed_type                              object
Number of amenities                    int64
guests_included                        int64
price_per_extra_person                 int64
minimum_nights                         int64
number_of_reviews                      int64
number_days_btw_first_last_review      int64
review_scores_rating                 float64
cancellation_policy                   object
dtype: object

In [11]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [12]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['host_is_superhost', 'host_identity_verified']

In [13]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [ ]:
binary_columns

In [ ]:
numeric_columns

In [ ]:
categorical_columns

# Pipeline

In [14]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [15]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [16]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [17]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [18]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-2.10940159, -1.39824237,  1.20477863, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.61906783, -1.38593382, -1.16133947, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.14448465, -0.16705969, -1.16133947, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82039585,  0.74441303, -0.56980994, ...,  0.        ,
         0.        ,  0.        ],
       [-0.92762441,  0.3821493 , -0.56980994, ...,  0.        ,
         0.        ,  1.        ],
       [-0.34071414, -0.53929512, -1.16133947, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
train_x.shape

# Tranform: transform() for TEST

In [19]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.63069768,  0.40533687,  1.79630816, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.15153485,  0.27611111, -0.56980994, ...,  0.        ,
         0.        ,  1.        ],
       [-2.02789334, -0.91924215,  0.02171958, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.15906806, -0.38872897,  1.20477863, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.11838687, -0.56878308, -0.56980994, ...,  0.        ,
         0.        ,  0.        ],
       [-0.94171792,  0.19283558, -1.16133947, ...,  0.        ,
         0.        ,  1.        ]])

In [20]:
test_x.shape

(3082, 61)

# SVM - Binary classification

## Baseline Accuracy

In [21]:
# Find majority class
train_y.value_counts()

price_gte_150
0                3661
1                3529
dtype: int64

In [22]:
# Find percentage
train_y.value_counts()/len(train_y)

price_gte_150
0                0.509179
1                0.490821
dtype: float64

## LinearSVC

LinearSVC fits a linear support vector machine classifier. It doesn't support kernel tricks. <br>

This is the same as `SVC(kernel='linear')` however, LinearSVC is more efficient when you have a large data set.

In [23]:
from sklearn.svm import LinearSVC 

# C is the margin width
# You can select l1 or l2 penalty using the penalty term. l2 is the default setting


svm_clf = LinearSVC(C=10)

svm_clf.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=10)

## Accuracy

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
#Predict the train values
train_y_pred = svm_clf.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8694019471488178

In [26]:
#Predict the test values
test_y_pred = svm_clf.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8598312783906554

## Classification Matrix

In [27]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[1272,  278],
       [ 154, 1378]], dtype=int64)

## Classification Report

In [28]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.89      0.82      0.85      1550
           1       0.83      0.90      0.86      1532

    accuracy                           0.86      3082
   macro avg       0.86      0.86      0.86      3082
weighted avg       0.86      0.86      0.86      3082



## LinearSVC with polynomial terms

In [29]:
from sklearn.preprocessing import PolynomialFeatures

# Create second degree terms
poly_features = PolynomialFeatures(degree=2, include_bias=False)

train_x_poly = poly_features.fit_transform(train_x)

#Don't forget to transform the test set
test_x_poly = poly_features.transform(test_x)

# If degree=2, then it creates all combinations: a, a^2, b, b^2, a.b, a^2.b, a.b^2, a^2.b^2 

In [36]:
#Fails to converge

pol_svm = LinearSVC(C=1, max_iter=10000)

pol_svm.fit(train_x_poly, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=1, max_iter=10000)

In [37]:
#Predict the train values
train_y_poly_pred = pol_svm.predict(train_x_poly)

#Train accuracy
accuracy_score(train_y, train_y_poly_pred)

0.9446453407510431

In [38]:
#Predict the test values
test_y_poly_pred = pol_svm.predict(test_x_poly)

#Test accuracy
accuracy_score(test_y, test_y_poly_pred)

0.9130434782608695

## SVC(kernel='linear')

This is the same as LinearSVC except it is slower and less efficient. However, it enables you to use other kernels such as "poly" or "rbf".

In [39]:
from sklearn.svm import SVC
 
lin_svm2 = SVC(kernel="linear")

lin_svm2.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

In [40]:
#Predict the train values
train_y_pred = lin_svm2.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8698191933240612

In [41]:
#Predict the test values
test_y_pred = lin_svm2.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8617780661907852

# SVC(kernel='poly') 

This is similar to running LinearSVC with polynomial terms. Though, this is much faster and more efficient.

In [48]:
from sklearn.svm import SVC

# You need to enter a value for gamma. Remember, gamma controls the shape of the bell curve for rbf
# You can also set it is as gamma='scale'. This will be the default option in future releases

pol_svm2 = SVC(kernel="poly", degree=3, coef0=1, C=10, gamma='scale')

pol_svm2.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=10, coef0=1, kernel='poly')

In [49]:
#Predict the train values
train_y_pred = pol_svm2.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.9677329624478442

In [50]:
#Predict the test values
test_y_pred = pol_svm2.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.9428942245295263

# SVC(kernel='rbf')

This is the Gaussian RBF.

In [51]:
rbf_svm = SVC(kernel="rbf", C=10, gamma='scale')

rbf_svm.fit(train_x, train_y)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=10)

In [52]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.9620305980528512

In [53]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.9367293964957819

# Multi Class Classification


## LinearSVC


In [54]:
train_set[['price_category']].head(10)

,price_category
3437,btw_75-150
6622,lte_75
2262,btw_75-150
2246,btw_75-150
835,gte_226
8618,btw_75-150
3955,gte_226
432,btw_75-150
3782,lte_75
6208,lte_75


In [55]:
# Assign new target variable
train_y_multiclass = train_set[['price_category']]
test_y_multiclass = test_set[['price_category']]

## Baseline

In [56]:
train_y_multiclass.value_counts()/len(train_y_multiclass)

price_category
btw_75-150        0.332823
btw_151-225       0.241725
lte_75            0.214743
gte_226           0.210709
dtype: float64

## LinearSVC

In [57]:
from sklearn.svm import LinearSVC 

#C is the margin width


svm_clf = LinearSVC(C=10, multi_class='ovr')

svm_clf.fit(train_x, train_y_multiclass)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=10)

In [58]:
#Predict the train values
train_y_pred = svm_clf.predict(train_x)

#Train accuracy
accuracy_score(train_y_multiclass, train_y_pred)

0.6148817802503477

In [59]:
#Predict the test values
test_y_pred = svm_clf.predict(test_x)

#Test accuracy
accuracy_score(test_y_multiclass, test_y_pred)

0.6194029850746269

In [60]:
#We usually create the confusion matrix on test set
confusion_matrix(test_y_multiclass, test_y_pred)

array([[399, 137, 146,  22],
       [227, 649,  40, 183],
       [179,  61, 431,   4],
       [  3, 170,   1, 430]], dtype=int64)

## SVC(kernel='poly')

In [61]:
pol_svm2 = SVC(kernel="poly", degree=2, coef0=1, C=5, gamma='scale', decision_function_shape='ovr')

pol_svm2.fit(train_x, train_y_multiclass)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=5, coef0=1, degree=2, kernel='poly')

In [62]:
#Predict the train values
train_y_pred = pol_svm2.predict(train_x)

#Train accuracy
accuracy_score(train_y_multiclass, train_y_pred)

0.7760778859527121

In [63]:
#Predict the test values
test_y_pred = pol_svm2.predict(test_x)

#Test accuracy
accuracy_score(test_y_multiclass, test_y_pred)

0.7193380921479559

In [64]:
#We usually create the confusion matrix on test set
confusion_matrix(test_y_multiclass, test_y_pred)

array([[487,  91, 114,  12],
       [152, 772,  27, 148],
       [165,  32, 477,   1],
       [  6, 117,   0, 481]], dtype=int64)

## SVC(kernel='rbf')

In [65]:
rbf_svm = SVC(kernel="rbf", C=10, gamma=0.1, decision_function_shape='ovr')

rbf_svm.fit(train_x, train_y_multiclass)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=10, gamma=0.1)

In [66]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y_multiclass, train_y_pred)

0.9595271210013908

In [67]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y_multiclass, test_y_pred)

0.899740428293316

In [68]:
#We usually create the confusion matrix on test set
confusion_matrix(test_y_multiclass, test_y_pred)

array([[ 611,   34,   58,    1],
       [  46, 1007,   13,   33],
       [  47,   12,  616,    0],
       [   6,   59,    0,  539]], dtype=int64)

# Grid Search

In [69]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 4 (2×2) combinations of hyperparameters
    {'C': [5, 15], 
     'gamma': [0.1, 0.2]}
  ]

rbf_svm = SVC(kernel="rbf", decision_function_shape='ovr')

# train across 5 folds, that's a total of 4*5=20 rounds of training 
grid_search = GridSearchCV(rbf_svm, param_grid, cv=5,
                           scoring='accuracy', return_train_score=True)

grid_search.fit(train_x, train_y_multiclass)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversio

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [5, 15], 'gamma': [0.1, 0.2]}],
             return_train_score=True, scoring='accuracy')

The best hyperparameter combination found:

In [70]:
grid_search.best_params_

{'C': 15, 'gamma': 0.2}

In [71]:
grid_search.best_estimator_

SVC(C=15, gamma=0.2)

Let's look at the score of each hyperparameter combination tested during the grid search:

In [72]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8478442280945758 {'C': 5, 'gamma': 0.1}
0.889847009735744 {'C': 5, 'gamma': 0.2}
0.884144645340751 {'C': 15, 'gamma': 0.1}
0.9047287899860917 {'C': 15, 'gamma': 0.2}


In [73]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,1.135820,0.009745,0.544588,0.013137,5,0.1,"{'C': 5, 'gamma': 0.1}",0.853268,0.847010,0.836579,...,0.847844,0.007699,4,0.933067,0.936718,0.936370,0.938804,0.935501,0.936092,0.001861
1,1.290381,0.049724,0.518976,0.004967,5,0.2,"{'C': 5, 'gamma': 0.2}",0.899861,0.891516,0.876217,...,0.889847,0.010376,2,0.971140,0.973748,0.975661,0.974965,0.973748,0.973853,0.001542
2,1.134809,0.010850,0.499093,0.007267,15,0.1,"{'C': 15, 'gamma': 0.1}",0.899166,0.882476,0.878999,...,0.884145,0.009127,3,0.965751,0.965403,0.968707,0.967837,0.964186,0.966377,0.001655
3,1.299893,0.040291,0.495860,0.001858,15,0.2,"{'C': 15, 'gamma': 0.2}",0.919332,0.900556,0.895688,...,0.904729,0.012165,1,0.985570,0.986613,0.989221,0.987830,0.987656,0.987378,0.001227


# Grid Search: randomized

In [74]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import random

param_distribs = {
        'C': randint(low=5, high=50),
        'gamma': uniform(0.1, 0.5),    
    }

rbf_svm = SVC(kernel="rbf", decision_function_shape='ovr')

rbf_search = RandomizedSearchCV(rbf_svm, param_distributions=param_distribs,
                                n_iter=5, cv=5, scoring='accuracy', random_state=42)

rbf_search.fit(train_x, train_y_multiclass)

c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nicol\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversio

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=5,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000201301BF130>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000201301BF700>},
                   random_state=42, scoring='accuracy')

In [75]:
cvres = rbf_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.909596662030598 {'C': 43, 'gamma': 0.49827149343011645}
0.9094575799721836 {'C': 19, 'gamma': 0.4659969709057026}
0.9109874826147426 {'C': 25, 'gamma': 0.17800932022121826}
0.9034770514603616 {'C': 23, 'gamma': 0.14998745790900145}
0.9061196105702365 {'C': 15, 'gamma': 0.5330880728874676}


## Run the final model on the Test Set

In [76]:
final_model = grid_search.best_estimator_

test_predictions = final_model.predict(test_x)

#Test accuracy
accuracy_score(test_y_multiclass, test_predictions)

0.9484101232965607

In [ ]:
# numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()
# categorical_columns = train_inputs.select_dtypes('object').columns.to_list()
# binary_columns = ['host_is_superhost', 'host_identity_verified']

In [ ]:
# Automate the binaries away
binary_cols = []
for column in numeric_columns:
    if len(pd.unique[airbnb[column]]) <= 2:
        binary_cols.append(cols)
        numeric_columns.remove(column)